In [1]:
# read new data
# read each page from excel file
import pandas as pd
import numpy as np

filename='cleaned'
Pacific=pd.read_excel(filename+'.xlsx',sheet_name='Pacific')
Atlantic=pd.read_excel(filename+'.xlsx',sheet_name='Atlantic')
Mediterranean=pd.read_excel(filename+'.xlsx',sheet_name='Mediterranean')
Southern=pd.read_excel(filename+'.xlsx',sheet_name='Southern Ocean')
Arctic=pd.read_excel(filename+'.xlsx',sheet_name='Arctic')

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
# we will be only merge one ocean at a time 

df2 = Pacific[['Lat','Long','Year']]
# convert to 'Lat','Long' string 
df2['Lat'] = df2['Lat'].astype(str)
df2['Long'] = df2['Long'].astype(str)

# here shockingly we have chinese character in the data!
df2['Lat'] = df2['Lat'].apply(lambda x: x.replace('−', '-')).astype(float)
df2['Long'] = df2['Long'].apply(lambda x: x.replace('−', '-')).astype(float)
# drop nan in lat and long 
df2 = df2.dropna(subset=['Lat','Long'])


/var/folders/4f/0qgxggzd35j3d8dc232z56440000gn/T/ipykernel_20178/1343268829.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Lat'] = df2['Lat'].astype(str)
/var/folders/4f/0qgxggzd35j3d8dc232z56440000gn/T/ipykernel_20178/1343268829.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Long'] = df2['Long'].astype(str)
/var/folders/4f/0qgxggzd35j3d8dc232z56440000gn/T/ipykernel_20178/1343268829.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usin

In [3]:
import pandas as pd
import tarfile
import gzip
from pyhdf.SD import SD, SDC
import tempfile
import numpy as np
from scipy.interpolate import griddata

# List of all the years
years = range(1997, 2024)

# Empty DataFrame to store all results
all_results = pd.DataFrame()

# Get the original latitudes and longitudes
orig_lats = df2['Lat'].copy()
orig_lons = df2['Long'].copy()

for year in years:
    with tarfile.open(f'data/par.s.{year}.tar', "r:") as tar:
        for member in tar.getmembers():
            if member.name.endswith('.hdf.gz'):
                # Decide month from the filename=day of year
                # last three integers
                day_of_year = int(str(member.name.split('.')[1])[-3:])
                month = str((day_of_year - 1) // 30 + 1).zfill(2)

                # Open the .hdf.gz file
                f = tar.extractfile(member)
                with gzip.open(f, 'rb') as gz:
                    # Decompress the .hdf.gz file to a temporary file
                    with tempfile.NamedTemporaryFile() as tmp:
                        tmp.write(gz.read())
                        tmp.seek(0)  # Go back to the start of the file

                        # Open the temporary .hdf file
                        hdf_file = SD(tmp.name, SDC.READ)
                        # Access the 'npp' dataset
                        data = hdf_file.select('par')[:]

                        # Replace '-9999.0' with NaN
                        data[data == -9999.0] = np.nan

                        # Define the latitude and longitude arrays
                        lats = np.linspace(90, -90, data.shape[0])  # Shape[0] is the number of rows
                        lons = np.linspace(-180, 180, data.shape[1])  # Shape[1] is the number of columns

                        lon_grid, lat_grid = np.meshgrid(lons, lats)
                        lat_list = lat_grid.reshape(-1,1)
                        lon_list = lon_grid.reshape(-1,1)
                        points = np.concatenate([lat_list, lon_list], axis=1)
                        xi = np.array(list(zip(df2['Lat'], df2['Long'])))
                        df2[f'par_{month}'] = griddata(points, data.ravel(), xi, method='nearest')

        # Add the year column
        df2['Year'] = year
        df2['Lat'] = orig_lats
        df2['Long'] = orig_lons

        # Append df2 to the results DataFrame
        all_results = pd.concat([all_results, df2])

    # Close the file
    hdf_file.end()

# save the data
all_results.to_csv('par_means_1.csv', index=False)

In [4]:
# add a column of yearly mean
all_results['yearly_mean_par'] = all_results.filter(regex='par_').mean(axis=1)

In [5]:
# add a column of Cr_nmol/kg from the 'cleaned' excel file
all_results['Cr_nmol/kg'] = Pacific['Cr_nmol/kg']

In [6]:
# create the 'Ocean' column filled with "Pacific" to the df_ocean_processed DataFrame
all_results['Ocean'] = 'Pacific'

In [7]:
# save the data csv
all_results.to_csv('par_means_1.csv', index=False)

In [8]:
## check the data
df_par_means = pd.read_csv('par_means_1.csv')
df_par_means.sample(10)

,Lat,Long,Year,par_10,par_11,par_12,par_01,par_02,par_04,par_05,par_06,par_07,par_08,par_09,par_03,yearly_mean_par,Cr_nmol/kg,Ocean
9671,21.476667,-126.595000,2015,43.563354,30.265165,25.576950,28.543900,48.103226,51.524967,52.658382,50.613980,50.346275,51.223625,48.067580,43.436516,43.660328,3.065813,Pacific
14012,24.283333,-114.983333,2023,42.392548,35.729366,30.058996,34.886550,50.883163,53.515568,48.803883,52.457397,57.861660,52.007725,47.847650,41.399918,45.653700,2.595384,Pacific
6682,46.000000,-158.000000,2009,13.996820,10.336063,9.007836,8.566417,24.176481,29.687132,37.433030,43.771500,35.645190,31.108965,24.761683,24.246418,24.394796,3.080000,Pacific
614,24.283333,-114.983333,1998,40.360960,33.517483,28.484533,30.729593,44.247433,53.067700,52.799720,56.900566,52.312920,52.691704,46.944910,NaN,44.732506,2.741385,Pacific
6416,-14.000000,-99.000000,2008,52.646350,49.477196,57.983517,55.068720,54.780950,47.975530,41.874190,35.072770,36.830692,40.945404,48.442265,52.546062,47.803640,2.997392,Pacific
2869,46.071667,-129.928333,2002,20.309692,11.539831,6.599642,8.220115,23.994345,35.790867,45.739395,44.754710,45.832428,41.778934,32.093483,24.329458,28.415243,4.439024,Pacific
1476,-34.670000,154.550000,1999,42.891700,51.410990,51.152927,51.264618,37.401585,24.850351,22.412460,14.926084,18.127250,26.553587,35.294075,NaN,34.207787,3.750000,Pacific
7504,21.000000,-132.000000,2011,41.710384,29.976696,24.854902,30.439160,43.377160,44.727130,44.051098,48.882400,50.035355,42.145756,44.395580,40.688656,40.440353,NaN,Pacific
10640,-34.670000,154.550000,2016,46.290485,59.511734,59.187420,51.391903,51.658516,31.805063,23.117966,15.132629,19.427675,26.529966,31.337168,42.379513,38.147503,NaN,Pacific
13300,-30.000000,175.000000,2021,43.398933,51.341602,55.577160,58.992180,44.996517,32.357500,23.429804,21.201480,22.145530,30.605125,39.291798,44.816162,39.012817,3.183553,Pacific


In [9]:
# we will be only merge one ocean at a time 

df3 = Atlantic[['Lat','Long','Year']]
# convert to 'Lat','Long' string 
df3['Lat'] = df3['Lat'].astype(str)
df3['Long'] = df3['Long'].astype(str)

# here shockingly we have chinese character in the data!
df3['Lat'] = df3['Lat'].apply(lambda x: x.replace('−', '-')).astype(float)
df3['Long'] = df3['Long'].apply(lambda x: x.replace('−', '-')).astype(float)
# drop nan in lat and long 
df3 = df3.dropna(subset=['Lat','Long'])


/var/folders/4f/0qgxggzd35j3d8dc232z56440000gn/T/ipykernel_20178/3129857474.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['Lat'] = df3['Lat'].astype(str)
/var/folders/4f/0qgxggzd35j3d8dc232z56440000gn/T/ipykernel_20178/3129857474.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['Long'] = df3['Long'].astype(str)
/var/folders/4f/0qgxggzd35j3d8dc232z56440000gn/T/ipykernel_20178/3129857474.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usin

In [12]:
import pandas as pd
import tarfile
import gzip
from pyhdf.SD import SD, SDC
import tempfile
import numpy as np
from scipy.interpolate import griddata

# List of all the years
years = range(1997, 2024)

# Empty DataFrame to store all results
all_results = pd.DataFrame()

# Get the original latitudes and longitudes
orig_lats = df3['Lat'].copy()
orig_lons = df3['Long'].copy()

for year in years:
    with tarfile.open(f'data/par.s.{year}.tar', "r:") as tar:
        for member in tar.getmembers():
            if member.name.endswith('.hdf.gz'):
                # Decide month from the filename=day of year
                # last three integers
                day_of_year = int(str(member.name.split('.')[1])[-3:])
                month = str((day_of_year - 1) // 30 + 1).zfill(2)

                # Open the .hdf.gz file
                f = tar.extractfile(member)
                with gzip.open(f, 'rb') as gz:
                    # Decompress the .hdf.gz file to a temporary file
                    with tempfile.NamedTemporaryFile() as tmp:
                        tmp.write(gz.read())
                        tmp.seek(0)  # Go back to the start of the file

                        # Open the temporary .hdf file
                        hdf_file = SD(tmp.name, SDC.READ)
                        # Access the 'npp' dataset
                        data = hdf_file.select('par')[:]

                        # Replace '-9999.0' with NaN
                        data[data == -9999.0] = np.nan

                        # Define the latitude and longitude arrays
                        lats = np.linspace(90, -90, data.shape[0])  # Shape[0] is the number of rows
                        lons = np.linspace(-180, 180, data.shape[1])  # Shape[1] is the number of columns

                        lon_grid, lat_grid = np.meshgrid(lons, lats)
                        lat_list = lat_grid.reshape(-1,1)
                        lon_list = lon_grid.reshape(-1,1)
                        points = np.concatenate([lat_list, lon_list], axis=1)
                        xi = np.array(list(zip(df3['Lat'], df3['Long'])))
                        df3[f'par_{month}'] = griddata(points, data.ravel(), xi, method='nearest')

        # Add the year column
        df3['Year'] = year
        df3['Lat'] = orig_lats
        df3['Long'] = orig_lons

        # Append df2 to the results DataFrame
        all_results = pd.concat([all_results, df3])

    # Close the file
    hdf_file.end()

# save the data
all_results.to_csv('par_means_2.csv', index=False)

In [14]:
# add a column of yearly mean
all_results['yearly_mean_par'] = all_results.filter(regex='par_').mean(axis=1)
# add a column of Cr_nmol/kg from the 'cleaned' excel file
all_results['Cr_nmol/kg'] = Atlantic['Cr_nmol/kg']
# create the 'Ocean' column filled with "Pacific" to the df_ocean_processed DataFrame
all_results['Ocean'] = 'Atlantic'

In [15]:
# save the data csv
all_results.to_csv('par_means_2.csv', index=False)

In [16]:
# we will be only merge one ocean at a time 

df4 = Mediterranean[['Lat','Long','Year']]
# convert to 'Lat','Long' string 
df4['Lat'] = df4['Lat'].astype(str)
df4['Long'] = df4['Long'].astype(str)

# here shockingly we have chinese character in the data!
df4['Lat'] = df4['Lat'].apply(lambda x: x.replace('−', '-')).astype(float)
df4['Long'] = df4['Long'].apply(lambda x: x.replace('−', '-')).astype(float)
# drop nan in lat and long 
df4 = df4.dropna(subset=['Lat','Long'])


/var/folders/4f/0qgxggzd35j3d8dc232z56440000gn/T/ipykernel_20178/1145701115.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4['Lat'] = df4['Lat'].astype(str)
/var/folders/4f/0qgxggzd35j3d8dc232z56440000gn/T/ipykernel_20178/1145701115.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df4['Long'] = df4['Long'].astype(str)
/var/folders/4f/0qgxggzd35j3d8dc232z56440000gn/T/ipykernel_20178/1145701115.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usin

In [17]:
# List of all the years
years = range(1997, 2024)

# Empty DataFrame to store all results
all_results = pd.DataFrame()

# Get the original latitudes and longitudes
orig_lats = df4['Lat'].copy()
orig_lons = df4['Long'].copy()

for year in years:
    with tarfile.open(f'data/par.s.{year}.tar', "r:") as tar:
        for member in tar.getmembers():
            if member.name.endswith('.hdf.gz'):
                # Decide month from the filename=day of year
                # last three integers
                day_of_year = int(str(member.name.split('.')[1])[-3:])
                month = str((day_of_year - 1) // 30 + 1).zfill(2)

                # Open the .hdf.gz file
                f = tar.extractfile(member)
                with gzip.open(f, 'rb') as gz:
                    # Decompress the .hdf.gz file to a temporary file
                    with tempfile.NamedTemporaryFile() as tmp:
                        tmp.write(gz.read())
                        tmp.seek(0)  # Go back to the start of the file

                        # Open the temporary .hdf file
                        hdf_file = SD(tmp.name, SDC.READ)
                        # Access the 'npp' dataset
                        data = hdf_file.select('par')[:]

                        # Replace '-9999.0' with NaN
                        data[data == -9999.0] = np.nan

                        # Define the latitude and longitude arrays
                        lats = np.linspace(90, -90, data.shape[0])  # Shape[0] is the number of rows
                        lons = np.linspace(-180, 180, data.shape[1])  # Shape[1] is the number of columns

                        lon_grid, lat_grid = np.meshgrid(lons, lats)
                        lat_list = lat_grid.reshape(-1,1)
                        lon_list = lon_grid.reshape(-1,1)
                        points = np.concatenate([lat_list, lon_list], axis=1)
                        xi = np.array(list(zip(df4['Lat'], df4['Long'])))
                        df4[f'par_{month}'] = griddata(points, data.ravel(), xi, method='nearest')

        # Add the year column
        df4['Year'] = year
        df4['Lat'] = orig_lats
        df4['Long'] = orig_lons

        # Append df2 to the results DataFrame
        all_results = pd.concat([all_results, df4])

    # Close the file
    hdf_file.end()

# save the data
all_results.to_csv('par_means_3.csv', index=False)

In [18]:
# add a column of yearly mean
all_results['yearly_mean_par'] = all_results.filter(regex='par_').mean(axis=1)
# add a column of Cr_nmol/kg from the 'cleaned' excel file
all_results['Cr_nmol/kg'] = Mediterranean['Cr_nmol/kg']
# create the 'Ocean' column filled with "Pacific" to the df_ocean_processed DataFrame
all_results['Ocean'] = 'Mediterranean'
# save the data csv
all_results.to_csv('par_means_3.csv', index=False)

In [19]:
# we will be only merge one ocean at a time 

df5 = Southern[['Lat','Long','Year']]
# convert to 'Lat','Long' string 
df5['Lat'] = df5['Lat'].astype(str)
df5['Long'] = df5['Long'].astype(str)

# here shockingly we have chinese character in the data!
df5['Lat'] = df5['Lat'].apply(lambda x: x.replace('−', '-')).astype(float)
df5['Long'] = df5['Long'].apply(lambda x: x.replace('−', '-')).astype(float)
# drop nan in lat and long 
df5 = df5.dropna(subset=['Lat','Long'])

/var/folders/4f/0qgxggzd35j3d8dc232z56440000gn/T/ipykernel_20178/2704579982.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df5['Lat'] = df5['Lat'].astype(str)
/var/folders/4f/0qgxggzd35j3d8dc232z56440000gn/T/ipykernel_20178/2704579982.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df5['Long'] = df5['Long'].astype(str)
/var/folders/4f/0qgxggzd35j3d8dc232z56440000gn/T/ipykernel_20178/2704579982.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usin

In [20]:
# List of all the years
years = range(1997, 2024)

# Empty DataFrame to store all results
all_results = pd.DataFrame()

# Get the original latitudes and longitudes
orig_lats = df5['Lat'].copy()
orig_lons = df5['Long'].copy()

for year in years:
    with tarfile.open(f'data/par.s.{year}.tar', "r:") as tar:
        for member in tar.getmembers():
            if member.name.endswith('.hdf.gz'):
                # Decide month from the filename=day of year
                # last three integers
                day_of_year = int(str(member.name.split('.')[1])[-3:])
                month = str((day_of_year - 1) // 30 + 1).zfill(2)

                # Open the .hdf.gz file
                f = tar.extractfile(member)
                with gzip.open(f, 'rb') as gz:
                    # Decompress the .hdf.gz file to a temporary file
                    with tempfile.NamedTemporaryFile() as tmp:
                        tmp.write(gz.read())
                        tmp.seek(0)  # Go back to the start of the file

                        # Open the temporary .hdf file
                        hdf_file = SD(tmp.name, SDC.READ)
                        # Access the 'npp' dataset
                        data = hdf_file.select('par')[:]

                        # Replace '-9999.0' with NaN
                        data[data == -9999.0] = np.nan

                        # Define the latitude and longitude arrays
                        lats = np.linspace(90, -90, data.shape[0])  # Shape[0] is the number of rows
                        lons = np.linspace(-180, 180, data.shape[1])  # Shape[1] is the number of columns

                        lon_grid, lat_grid = np.meshgrid(lons, lats)
                        lat_list = lat_grid.reshape(-1,1)
                        lon_list = lon_grid.reshape(-1,1)
                        points = np.concatenate([lat_list, lon_list], axis=1)
                        xi = np.array(list(zip(df5['Lat'], df5['Long'])))
                        df5[f'par_{month}'] = griddata(points, data.ravel(), xi, method='nearest')

        # Add the year column
        df5['Year'] = year
        df5['Lat'] = orig_lats
        df5['Long'] = orig_lons

        # Append df2 to the results DataFrame
        all_results = pd.concat([all_results, df5])

    # Close the file
    hdf_file.end()

# save the data
all_results.to_csv('par_means_4.csv', index=False)

In [21]:
# add a column of yearly mean
all_results['yearly_mean_par'] = all_results.filter(regex='par_').mean(axis=1)
# add a column of Cr_nmol/kg from the 'cleaned' excel file
all_results['Cr_nmol/kg'] = Southern['Cr_nmol/kg']
# create the 'Ocean' column filled with "Pacific" to the df_ocean_processed DataFrame
all_results['Ocean'] = 'Southern'
# save the data csv
all_results.to_csv('par_means_4.csv', index=False)

In [22]:
# we will be only merge one ocean at a time 

df6 = Arctic[['Lat','Long','Year']]
# convert to 'Lat','Long' string 
df6['Lat'] = df6['Lat'].astype(str)
df6['Long'] = df6['Long'].astype(str)

# here shockingly we have chinese character in the data!
df6['Lat'] = df6['Lat'].apply(lambda x: x.replace('−', '-')).astype(float)
df6['Long'] = df6['Long'].apply(lambda x: x.replace('−', '-')).astype(float)
# drop nan in lat and long 
df6 = df6.dropna(subset=['Lat','Long'])

/var/folders/4f/0qgxggzd35j3d8dc232z56440000gn/T/ipykernel_20178/1638750860.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df6['Lat'] = df6['Lat'].astype(str)
/var/folders/4f/0qgxggzd35j3d8dc232z56440000gn/T/ipykernel_20178/1638750860.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df6['Long'] = df6['Long'].astype(str)
/var/folders/4f/0qgxggzd35j3d8dc232z56440000gn/T/ipykernel_20178/1638750860.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usin

In [23]:
# List of all the years
years = range(1997, 2024)

# Empty DataFrame to store all results
all_results = pd.DataFrame()

# Get the original latitudes and longitudes
orig_lats = df6['Lat'].copy()
orig_lons = df6['Long'].copy()

for year in years:
    with tarfile.open(f'data/par.s.{year}.tar', "r:") as tar:
        for member in tar.getmembers():
            if member.name.endswith('.hdf.gz'):
                # Decide month from the filename=day of year
                # last three integers
                day_of_year = int(str(member.name.split('.')[1])[-3:])
                month = str((day_of_year - 1) // 30 + 1).zfill(2)

                # Open the .hdf.gz file
                f = tar.extractfile(member)
                with gzip.open(f, 'rb') as gz:
                    # Decompress the .hdf.gz file to a temporary file
                    with tempfile.NamedTemporaryFile() as tmp:
                        tmp.write(gz.read())
                        tmp.seek(0)  # Go back to the start of the file

                        # Open the temporary .hdf file
                        hdf_file = SD(tmp.name, SDC.READ)
                        # Access the 'npp' dataset
                        data = hdf_file.select('par')[:]

                        # Replace '-9999.0' with NaN
                        data[data == -9999.0] = np.nan

                        # Define the latitude and longitude arrays
                        lats = np.linspace(90, -90, data.shape[0])  # Shape[0] is the number of rows
                        lons = np.linspace(-180, 180, data.shape[1])  # Shape[1] is the number of columns

                        lon_grid, lat_grid = np.meshgrid(lons, lats)
                        lat_list = lat_grid.reshape(-1,1)
                        lon_list = lon_grid.reshape(-1,1)
                        points = np.concatenate([lat_list, lon_list], axis=1)
                        xi = np.array(list(zip(df6['Lat'], df6['Long'])))
                        df6[f'par_{month}'] = griddata(points, data.ravel(), xi, method='nearest')

        # Add the year column
        df6['Year'] = year
        df6['Lat'] = orig_lats
        df6['Long'] = orig_lons

        # Append df2 to the results DataFrame
        all_results = pd.concat([all_results, df6])

    # Close the file
    hdf_file.end()

# save the data
all_results.to_csv('par_means_5.csv', index=False)

In [24]:
# add a column of yearly mean
all_results['yearly_mean_par'] = all_results.filter(regex='par_').mean(axis=1)
# add a column of Cr_nmol/kg from the 'cleaned' excel file
all_results['Cr_nmol/kg'] = Arctic['Cr_nmol/kg']
# create the 'Ocean' column filled with "Pacific" to the df_ocean_processed DataFrame
all_results['Ocean'] = 'Arctic'
# save the data csv
all_results.to_csv('par_means_5.csv', index=False)

In [25]:
dt1 = pd.read_csv('par_means_1.csv')
dt2 = pd.read_csv('par_means_2.csv')
dt3 = pd.read_csv('par_means_3.csv')
dt4 = pd.read_csv('par_means_4.csv')
dt5 = pd.read_csv('par_means_5.csv')
merged_df = pd.concat([dt1, dt2, dt3, dt4, dt5])
merged_df.to_csv('par_means_new.csv', index=False)


In [26]:
## check the data
df_par_means = pd.read_csv('par_means_new.csv')
df_par_means.sample(10)

,Lat,Long,Year,par_10,par_11,par_12,par_01,par_02,par_04,par_05,par_06,par_07,par_08,par_09,par_03,yearly_mean_par,Cr_nmol/kg,Ocean
25595,-53.200000,-118.130000,2011,23.221207,33.755898,40.123245,38.709644,18.477451,12.326264,5.835998,3.655497,4.281707,10.785999,17.938750,21.155030,19.188890,3.610000,Southern
24441,-53.200000,-118.130000,2003,29.996048,42.178066,45.970646,39.250920,22.219078,12.870097,6.004094,4.003114,4.463336,9.028788,15.881397,20.608252,21.039484,3.560000,Southern
22030,12.588200,-17.572400,2021,41.133095,44.669000,41.831516,45.489967,55.651886,56.675297,55.496883,49.924030,48.094180,38.391980,48.538750,55.832714,48.477436,2.209834,Atlantic
6959,-14.000000,-99.000000,2009,49.378242,49.663567,52.415276,58.580124,51.227787,47.738200,40.054436,36.618750,38.746708,39.270485,46.376514,52.546062,46.884678,3.290217,Pacific
20144,54.500000,-48.460000,2015,10.800384,6.046431,2.643207,2.981126,16.395740,24.596098,33.646580,40.066520,40.962986,29.739094,20.107464,15.257352,20.270248,3.561254,Atlantic
276,24.167167,-160.253833,1997,42.051144,30.994768,30.020460,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34.355457,2.980000,Pacific
9477,20.000000,-106.000000,2014,46.146694,32.922665,35.216774,38.659550,53.223454,56.529034,55.485367,50.392082,53.701256,53.961227,48.619278,51.740130,48.049790,2.290000,Pacific
25156,-54.830000,-95.680000,2008,25.020788,39.678770,34.635914,44.185596,32.300150,9.656915,5.138465,2.714197,4.511142,7.979820,18.968130,20.821610,20.467623,3.580000,Southern
10239,23.485000,-117.021667,2016,40.980595,33.403683,25.773417,35.674020,40.748740,52.316666,51.984970,48.914066,57.904350,48.172030,45.598133,45.837357,43.942337,2.698357,Pacific
3845,24.698333,-114.106667,2004,43.194904,34.818195,28.147224,31.711159,40.731240,48.593280,56.947790,55.246437,54.605484,56.805710,48.676662,42.518837,45.166412,2.950474,Pacific
